# Course project


## **Основное**
- Дедлайн - 4 апреля 23:59
- Целевая метрика precision@5
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. В решении должны быть отчетливо видна метрика на новом тестовом сете из файла retail_test1.csv, то есть вам нужно для всех юзеров из этого файла выдать выши рекомендации, и посчитать на actual покупках precision@5. 


In [ ]:
# !pip install implicit==0.4.4

## Импорт необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

## Определение функций и классов

In [2]:
# функция показывающая сколько пользователей и товаров в передатнном датасете
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}")

# функция добавляющая рекомендации в результирующий датасет для оценки моделей
# recommend_models - список моделей для выдачи рекомендаций и последующей оценке их работы
def myEvalRecall(df_result, target_col_name, recommend_models, n_predict = 50, topk_recall = 50):
    for model in recommend_models:
        result_col_name = model
        df_result[result_col_name] = df_result[target_col_name].apply(lambda x: recommend_models[model](x, N=n_predict))
    return sorted(calc_recall(df_result, topk_recall), key=lambda x: x[1],reverse=True)

# функция добавляющая рекомендации в результирующий датасет для оценки моделей
def evalRecall(df_result, target_col_name, recommend_model, n_predict = 50, topk_recall = 50):
    result_col_name = 'result'
    df_result[result_col_name] = df_result[target_col_name].apply(lambda x: recommend_model(x, N=n_predict))
    return df_result.apply(lambda row: recall_at_k(row[result_col_name], row[ACTUAL_COL], k=topk_recall), axis=1).mean()

# df_result.apply(lambda row: recall_at_k(row[result_col_name], row[ACTUAL_COL], k=n_predict), axis=1).mean()

# функция расчета метрики RECALL@K
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

# функция расчета метрики  PRECISION@K     
def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

# функция ранжирования на датасете df_ranker_predict (по результату работы классификатора)
def rerank(user_id):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id].sort_values('proba_item_purchase', ascending=False).head(5).item_id.tolist()

## Загрузка и подготовка данных

In [3]:
# загружаем данные
PATH_DATA = "../data"
data = pd.read_csv(os.path.join(PATH_DATA,'retail_train.csv'))
item_features = pd.read_csv(os.path.join(PATH_DATA,'product.csv'))
user_features = pd.read_csv(os.path.join(PATH_DATA,'hh_demographic.csv'))

In [4]:
# Определяем константы
ITEM_COL = 'item_id'
USER_COL = 'user_id'
ACTUAL_COL = 'actual'

# Количество рекомендуемых кандидатов
N_PREDICT = 500 
# Количество элементов для оценки в метрике recall
TOPK_RECALL = 500
# Количество элементов для оценки в метрике precision
TOPK_PRECISION = 5

In [5]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

In [6]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 3 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
# размер второго датасета установим в 3 недели ( по итогам исследований в ДЗ 6)

VAL_MATCHER_WEEKS = 3
VAL_RANKER_WEEKS = 3

## Разбиваем данные на train, eval, test

In [7]:
# берем данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

# берем данные для валидации matching модели
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]

# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

In [8]:
# сделаем объединенный сет данных для первого уровня (матчинга)
df_join_train_matcher = pd.concat([data_train_matcher, data_val_matcher])

In [9]:
print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (2193515, 12) Users: 2499 Items: 85334
val_matcher
Shape: (84975, 12) Users: 1889 Items: 20567
train_ranker
Shape: (84975, 12) Users: 1889 Items: 20567
val_ranker
Shape: (118314, 12) Users: 2042 Items: 24329


In [10]:
data_val_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2191387,84,40877069294,615,862732,1,0.88,382,-0.71,8,89,0.0,0.0
2191388,84,40877069294,615,920025,1,5.99,382,0.00,8,89,0.0,0.0


## Фильтрация данных

In [11]:
n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/Users/eugene/Документы/Education/AI/Рекомендательные системы/course_project/utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 85334 to 5001


In [12]:
# делаем так, чтобы все датасеты работали с известными пользователями
common_users = data_train_matcher.user_id.values

data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (896610, 13) Users: 2496 Items: 5001
val_matcher
Shape: (84953, 12) Users: 1887 Items: 20565
train_ranker
Shape: (84953, 12) Users: 1887 Items: 20565
val_ranker
Shape: (118282, 12) Users: 2040 Items: 24325


# Init/train recommender

In [13]:
# создаем и обучаем модели для получения кандидатов для рекомендаций
recommender = MainRecommender(data_train_matcher)

# Eval recall of matching

In [14]:
# создадим базовую таблицу для последующей оценки моделей по рекомендованным кандидатам
# для этого используем валидационный датасет (второй, VAL_MATCHER_WEEKS = 2 недели)
result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,1,"[829323, 835108, 836423, 851515, 875240, 87737..."
1,2,"[895388, 8357614, 12301772, 821083, 828106, 83..."


### Recall@50 of matching

In [15]:
print(f'Количество рекомендуемых кандидатов: {N_PREDICT}' )
print(f'Количество элементов для оценки в метрике recall: {TOPK_RECALL}')


Количество рекомендуемых кандидатов: 500
Количество элементов для оценки в метрике recall: 500


In [16]:
candidad_models = {'own_rec': recommender.get_own_recommendations, 
                   'sim_item_rec': recommender.get_similar_items_recommendation, 
                   'als_rec': recommender.get_als_recommendations}

myEvalRecall(result_eval_matcher, USER_COL, candidad_models, n_predict = N_PREDICT, topk_recall = TOPK_RECALL)

[('own_rec', 0.19296530930023),
 ('als_rec', 0.15938609159887246),
 ('sim_item_rec', 0.13957748813902454)]

### Precision@5 of matching

In [17]:
print(f'Количество элементов для оценки в метрике precision: {TOPK_PRECISION}')

Количество элементов для оценки в метрике precision: 5


In [18]:
sorted(calc_precision(result_eval_matcher, TOPK_PRECISION), key=lambda x: x[1],reverse=True)

[('own_rec', 0.13651298357180627),
 ('als_rec', 0.08224695283518818),
 ('sim_item_rec', 0.04610492845786987)]

# Ranking part

### Обучаем модель 2-ого уровня на выбранных кандидатах

- Обучаем на data_train_ranking
- Обучаем *только* на выбранных кандидатах
- Генерим топ-500 кадидиатов через get_own_recommendations
- (!) Если юзер купил < 500 товаров, то get_own_recommendations дополнит рекоммендации топ-популярными

In [19]:
# -- давние покупки -- | -- 3 недели -- | -- 3 недели -- 

## Подготовка данных для трейна

In [20]:
# взяли пользователей из трейна для ранжирования (3 недели)
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]

In [21]:
# собираем топ - 500 кандитатов с первого этапа (matcher)
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

In [22]:
df_match_candidates.head(2)

,user_id,candidates
0,84,"[903529, 920025, 977374, 829722, 987518, 87304..."
1,1753,"[967041, 963686, 948640, 1057168, 9553382, 942..."


In [23]:
# разворачиваем товары
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'

In [24]:
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

In [25]:
df_match_candidates.head(4)

,user_id,item_id
0,84,903529
0,84,920025
0,84,977374
0,84,829722


### Check warm start

In [26]:
print_stats_data(df_match_candidates, 'match_candidates')

match_candidates
Shape: (943500, 2) Users: 1887 Items: 4654


### Создаем трейн сет для ранжирования с учетом кандидатов с этапа 1 

In [27]:
# создаем базовый датафрейм для последующего ранжирования из датасета для ранжирования (3 недели)
# таргет для всех записей = 1 так как здесь только купленные пользователем товары
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # тут только покупки 

df_ranker_train.head()

,user_id,item_id,target
2191387,84,862732,1
2191388,84,920025,1
2191389,84,984283,1
2191390,84,1096226,1
2191391,84,1120258,1


In [28]:
# соединим созданный базовый датафрейм с полученными кандидатами
# если пара user - item новая, то для таких таргет поставим = 0 (мы конечно не знаем наверняка, но тем не менее)
df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])

df_ranker_train['target'].fillna(0, inplace= True)

In [29]:
df_ranker_train.target.value_counts()

0.0    871516
1.0     15324
Name: target, dtype: int64

In [31]:
df_ranker_train['target'].mean()

0.01727932885300618

## Подготавливаем фичи для обучения модели

### Описательные фичи

In [32]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [33]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [34]:
df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')

df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,84,903529,1.0,539,DRUG GM,National,CIGARETTES,CIGARETTES,972976 PK,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84,920025,1.0,764,GROCERY,National,LAUNDRY ADDITIVES,FABRIC SOFTENER LIQUID,60 LOAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Поведенческие фичи

##### Чтобы считать поведенческие фичи, нужно учесть все данные что были до data_val_ranker

In [36]:
df_join_train_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [37]:
# добавим средний чек пользователя к тренировочному датасету для ранжирования
df_ranker_train = df_ranker_train.merge((df_join_train_matcher.groupby(USER_COL).agg('sales_value').sum()/df_join_train_matcher.groupby(USER_COL).agg('quantity').sum()).rename('average_check'), how='left', on=USER_COL)

# добавим общую сумму продажи товара в тренировочный датасет для ранжирования
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('sales_value').sum().rename('total_item_sales_value'), how='left', on=ITEM_COL)

# добавим общее количество продаж каждого товара в тренировочный датасет для ранжирования
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('total_quantity_value'), how='left', on=ITEM_COL)

# добавим количество товаров, которые куплены пользователями
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('sales_value').sum().rename('total_user_sales_value'), how='left',on=USER_COL)


In [39]:
df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,...,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,average_check,total_item_sales_value,total_quantity_value,item_freq,user_freq,total_user_sales_value
0,84,903529,1.0,539,DRUG GM,National,CIGARETTES,CIGARETTES,972976 PK,NaN,...,NaN,NaN,NaN,NaN,0.125995,219.15,65,46,416,1088.09
1,84,920025,1.0,764,GROCERY,National,LAUNDRY ADDITIVES,FABRIC SOFTENER LIQUID,60 LOAD,NaN,...,NaN,NaN,NaN,NaN,0.125995,433.77,78,75,416,1088.09


## Обучение модели ранжирования

In [40]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [41]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

In [42]:
lgb = LGBMClassifier(objective='binary',
                     max_depth=10,
                     n_estimators=500,
                     learning_rate=0.1,
                     categorical_column=cat_feats
                    )

lgb.fit(X_train, y_train)

/Users/eugene/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/eugene/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


LGBMClassifier(categorical_column=['manufacturer', 'department', 'brand',
                                   'commodity_desc', 'sub_commodity_desc',
                                   'curr_size_of_product', 'age_desc',
                                   'marital_status_code', 'income_desc',
                                   'homeowner_desc', 'hh_comp_desc',
                                   'household_size_desc', 'kid_category_desc',
                                   'average_check', 'total_item_sales_value',
                                   'total_quantity_value', 'item_freq',
                                   'user_freq', 'total_user_sales_value'],
               max_depth=10, n_estimators=500, objective='binary')

In [43]:
train_preds = lgb.predict_proba(X_train)

In [44]:
df_ranker_predict = df_ranker_train.copy()

In [45]:
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

## Оценка на валидационном датасете для ранжирования

In [46]:
# создадим базовый датасет для оценки качества на валидационном датасете ранжирования (3 недели)
result_eval_ranker = data_val_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_ranker.columns=[USER_COL, ACTUAL_COL]
result_eval_ranker.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [47]:
%%time
# базовая рекомендация
result_eval_ranker['own_rec'] = result_eval_ranker[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))


CPU times: user 5.57 s, sys: 533 ms, total: 6.1 s
Wall time: 6.13 s


In [48]:
# ранжированная рекомендация + какие то настройки (дополнительные фичи и т.д.)
result_eval_ranker['reranked_own'] = result_eval_ranker[USER_COL].apply(lambda user_id: rerank(user_id))
# сравниваем метрику precision до и после ранжирования
print(*sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own', 0.24420080784766032)
('own_rec', 0.1495098039215674)


/Users/eugene/Документы/Education/AI/Рекомендательные системы/course_project/metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


## Подготовка данных и моделей для оценки на тесте

In [49]:
# Схема подбора кандидатов и ранжирования будет следующая
# -- давние покупки: подбор кандидатов -- | -- 3 недели: ранжирование -- | -- 3 недели: тест -- 
# тестовый датасет - 3 следующие недели, поэтому ранжируем на датасете максимально близком по времени -

VAL_RANKER_WEEKS = 3
# данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - VAL_RANKER_WEEKS]

# данные для тренировки ranking модели
data_train_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

# сделаем объединенный сет данных
df_join_train_matcher = pd.concat([data_train_matcher, data_train_ranker])

# фильтрация данных
n_items_before = data_train_matcher['item_id'].nunique()
data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=5000)
n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

# устаналиваем единый набор пользователей для всех датасетов
common_users = data_train_matcher.user_id.values
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_train_ranker,'train_ranker')

# обучаем модель для мэтчинга
recommender = MainRecommender(data_train_matcher)

# создаем базовый датафрейм для последующего ранжирования
# таргет для всех записей = 1 так как здесь только купленные пользователем товары
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # тут только покупки 

# взяли пользователей из датасета  для ранжирования 
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]
# собираем топ - 500 кандитатов с первого этапа 
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
# разворачиваем товары
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

# соединим созданный базовый датафрейм с полученными кандидатами
# если пара user - item новая, то для таких таргет поставим = 0 (мы конечно не знаем наверняка, но тем не менее)
df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')
# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])
df_ranker_train['target'].fillna(0, inplace= True)

# добавляем признаки для товаров и пользователей
df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')


# создаем дополнительные признаки
# добавим средний чек пользователя к тренировочному датасету для ранжирования
df_ranker_train = df_ranker_train.merge((df_join_train_matcher.groupby(USER_COL).agg('sales_value').sum()/df_join_train_matcher.groupby(USER_COL).agg('quantity').sum()).rename('average_check'), how='left', on=USER_COL)
# добавим общую сумму продажи товара в тренировочный датасет для ранжирования
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('sales_value').sum().rename('total_item_sales_value'), how='left', on=ITEM_COL)
# добавим общее количество продаж каждого товара в тренировочный датасет для ранжирования
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('total_quantity_value'), how='left', on=ITEM_COL)
# добавим количество товаров, которые куплены пользователями
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('sales_value').sum().rename('total_user_sales_value'), how='left',on=USER_COL)

# Обучаем модель классификации для ранжирования кандидатов
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')
lgb = LGBMClassifier(objective='binary',
                     max_depth=10,
                     n_estimators=500,
                     learning_rate=0.1,
                     categorical_column=cat_feats
                    )

lgb.fit(X_train, y_train)

# предсказываем результаты
train_preds = lgb.predict_proba(X_train)
df_ranker_predict = df_ranker_train.copy()
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]


/Users/eugene/Документы/Education/AI/Рекомендательные системы/course_project/utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 86865 to 5001
train_matcher
Shape: (931437, 13) Users: 2497 Items: 5001
train_ranker
Shape: (118282, 12) Users: 2040 Items: 24325


/Users/eugene/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/eugene/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


# Оценка на тесте

In [50]:
# загружаем тестовый датасет
df_test = pd.read_csv(os.path.join(PATH_DATA,'retail_test1.csv'))
df_test.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0


In [51]:
# как расположен тестовый датасет относительно датасетов для подбора кандидатов и ранжирования по времени
df_test.week_no.value_counts()

98    31153
97    29208
96    28373
Name: week_no, dtype: int64

То есть это следующие 3 недели после нашего датасета для ранжирования

In [52]:
# Проверим наличие новых пользователей в тестовом наборе
new_test_users_ids = set(df_test['user_id']) - set(data_train_matcher['user_id'])
len(new_test_users_ids)

2

In [53]:
# удалим появившихся новых пользователей, оставив только уже имевшихся в наших датасетах
df_test = df_test[df_test.user_id.isin(common_users)]

In [54]:
# создадим базовый датасет для оценки итоговой метрики
result_test = df_test.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_test.columns=[USER_COL, ACTUAL_COL]
result_test.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784..."


In [55]:

%%time
result_test['own_rec'] = result_test[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

CPU times: user 5.39 s, sys: 45.6 ms, total: 5.44 s
Wall time: 5.65 s


In [56]:
# ранжируем кандидатов используя результат работы модели второго этапа (классификатора)
result_test['reranked_own_rec'] = result_test[USER_COL].apply(lambda user_id: rerank(user_id))
print(*sorted(calc_precision(result_test, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.22582513028372625)
('own_rec', 0.13212958045671722)


/Users/eugene/Документы/Education/AI/Рекомендательные системы/course_project/metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


In [58]:
precision_list = sorted(calc_precision(result_test, TOPK_PRECISION))
precision_list              

[('own_rec', 0.13212958045671722), ('reranked_own_rec', 0.22582513028372625)]

# Целевая метрика precision@5

In [65]:
print(f'Целевая метрика precision@5: {round(precision_list[1][1], 4)}')

Целевая метрика precision@5: 0.2258
